# Add Cubes Around Surface of Precentral Gyrus

In [1]:
from freesurfer_surface import Surface

SUBJECTS_DIR = '../tests/subjects'
surface = Surface.read_triangular(SUBJECTS_DIR + '/fabian/surf/lh.pial')
surface.load_annotation_file(SUBJECTS_DIR + '/fabian/label/lh.aparc.annot')

precentral_label, = filter(lambda l: l.name == 'precentral', surface.annotation.labels.values())
precentral_label

Label(name=precentral, index=24, color=#3c14dc)

In [2]:
border_chain, = surface.find_label_border_polygonal_chains(precentral_label)
border_chain

PolygonalChain(vertex_indices=(33450, 33454, 34870, 36281, 37761, 39213, 39193, 40741, 42258, 43800, 43786, 43785, 45355, 45344, 45343, 46815, 46811, 48178, 49598, 51012, 51013, 52441, 53824, 55100, 56329, 57502, 58620, 58616, 59710, 60747, 60750, 60749, 61818, 62930, 62931, 62941, 62949, 62950, 62959, 62960, 64163, 64175, 64187, 64195, 64202, 64203, 64210, 65576, 65585, 66968, 66976, 66977, 66986, 66996, 67005, 67006, 67016, 67017, 67018, 67019, 67020, 68224, 68225, 69437, 70772, 72119, 73344, 74639, 74640, 75876, 77094, 78276, 78288, 79367, 80520, 80545, 80546, 80574, 80594, 79445, 78350, 78351, 79447, 79448, 79426, 79427, 79428, 79398, 79399, 79400, 78316, 79368, 79369, 79350, 79351, 80503, 80488, 81684, 81685, 82897, 82885, 84057, 84058, 84049, 85333, 86554, 86533, 86516, 86506, 87755, 87756, 89146, 90404, 90405, 91703, 92898, 92897, 91715, 91729, 91728, 91751, 91750, 91765, 91779, 90513, 90532, 90550, 89220, 89243, 89265, 87885, 87903, 87921, 87942, 87964, 87980, 89341, 90631, 918

In [3]:
import numpy
from freesurfer_surface import Vertex, Triangle

def unit_vector(vector: numpy.array) -> numpy.array:
    return vector / numpy.linalg.norm(vector)

BORDER_WIDTH = 0.8 # mm

for vertex_triplet_indices in border_chain.adjacent_vertex_indices(3):
    vertex_triplet = surface.select_vertices(vertex_triplet_indices)
    backward_vector = vertex_triplet[0] - vertex_triplet[1]
    forward_vector = unit_vector(vertex_triplet[2] - vertex_triplet[1]) * BORDER_WIDTH / 2
    upward_vector = unit_vector(numpy.cross(backward_vector, forward_vector)) * BORDER_WIDTH / 2
    sideward_vector = unit_vector(numpy.cross(upward_vector, forward_vector)) * BORDER_WIDTH / 2
    cross_section_corners = [vertex_triplet[1] + v
                             for v in [-upward_vector -sideward_vector,
                                       -upward_vector +sideward_vector,
                                       +upward_vector +sideward_vector,
                                       +upward_vector -sideward_vector]]
    corner_indices = [[surface.add_vertex(Vertex(*(coords + forward_dir))) for coords in cross_section_corners]
                      for forward_dir in [-forward_vector, +forward_vector]]
    for surface_corner_indices in corner_indices:
        surface.add_rectangle(surface_corner_indices)
    for i in range(4):
        j = (i + 1) % 4
        surface.add_rectangle((corner_indices[0][i], corner_indices[0][j],
                               corner_indices[1][j], corner_indices[1][i]))

surface.write_triangular('precentral-border.lh.pial')

```sh
freeview --surface examples/precentral-border.lh.pial:annot=tests/subjects/fabian/label/lh.aparc.annot
```